In [1]:
import pandas as pd

file_path = '/content/drive/MyDrive/Politician Trades/more_prices.csv'
df = pd.read_csv(file_path)
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df

,Politician Name,Party,Chamber,State,Issuer Name,Ticker,Publication Date,Transaction Date,Reporting Gap,Owner,...,Price_-90_days,Price_-60_days,Price_-40_days,Price_-20_days,Price_-10_days,Price_10_days,Price_20_days,Price_40_days,Price_60_days,Price_90_days
0,Tom Carper,Democrat,Senate,DE,Valero Energy Corp,VLO,2024-05-01,2024-04-11,19,Spouse,...,131.500000,142.529999,143.279999,169.639999,172.639999,165.899994,155.660004,NaN,NaN,NaN
1,Tom Carper,Democrat,Senate,DE,Enbridge Inc,ENB,2024-05-01,2024-04-11,19,Spouse,...,36.840000,34.889999,34.799999,35.450001,35.830002,35.349998,35.590000,NaN,NaN,NaN
2,Dan Newhouse,Republican,House,WA,Accenture PLC,ACN,2024-04-26,2024-04-10,15,Spouse,...,347.970001,371.670013,380.989990,345.029999,339.170013,316.880005,300.910004,NaN,NaN,NaN
3,Dan Newhouse,Republican,House,WA,Analog Devices Inc,ADI,2024-04-26,2024-04-10,15,Spouse,...,188.960007,195.020004,196.160004,195.149994,196.619995,183.360001,200.610001,NaN,NaN,NaN
4,Dan Newhouse,Republican,House,WA,Booking Holdings Inc,BKNG,2024-04-26,2024-04-10,15,Spouse,...,3552.280029,3758.179932,3499.729980,3647.810059,3568.870117,3414.820068,3452.030029,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31584,Alan Lowenthal,Democrat,House,CA,Americold Realty Trust,COLD,2021-03-24,2021-03-15,8,Spouse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31585,Mark Green,Republican,House,TN,NGL Energy Partners LP,NGL,2021-03-21,2021-03-17,3,Undisclosed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31586,Doug Lamborn,Republican,House,CO,NetApp Inc,NTAP,2021-03-20,2021-03-19,0,Spouse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31587,Doug Lamborn,Republican,House,CO,NetApp Inc,NTAP,2021-03-20,2021-03-19,0,Undisclosed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31589 entries, 0 to 31588
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Politician Name            31589 non-null  object        
 1   Party                      31589 non-null  object        
 2   Chamber                    31589 non-null  object        
 3   State                      31589 non-null  object        
 4   Issuer Name                31589 non-null  object        
 5   Ticker                     31589 non-null  object        
 6   Publication Date           31589 non-null  object        
 7   Transaction Date           31589 non-null  datetime64[ns]
 8   Reporting Gap              31589 non-null  int64         
 9   Owner                      31589 non-null  object        
 10  Transaction Type           31589 non-null  object        
 11  Value Range                31589 non-null  object        
 12  Pric

In [3]:
import pandas as pd
import yfinance as yf
from tqdm import tqdm

def fetch_closest_price(ticker_symbol, date):
    start_date = date - pd.Timedelta(days=3)
    end_date = date + pd.Timedelta(days=3)
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    try:
        data = yf.download(ticker_symbol, start=start_date_str, end=end_date_str, progress=False)
        data.dropna(subset=['Close'], inplace=True)
        data['date_diff'] = abs((data.index - date).days)
        data.sort_values('date_diff', inplace=True)
        return data.iloc[0]['Close'] if not data.empty else None
    except Exception as e:
        # print(f"Error downloading data for {ticker_symbol} on {date.date()}: {e}")
        return None

days_offsets = [-90, -60, -40, -20, -10, 10, 20, 40, 60, 90]

def process_data(start_index=22600):
    today = pd.Timestamp('today').normalize()

    for offset in days_offsets:
        col_name = f'Price_{offset}_days'
        if col_name not in df.columns:
            df[col_name] = pd.NA

    for index, row in tqdm(df.iloc[start_index:].iterrows(), total=(df.shape[0] - start_index), desc='Processing rows'):
        if (index - start_index) % 100 == 0:
            df.to_csv(file_path, index=False)

        ticker_symbol = row['Ticker']
        transaction_date = pd.to_datetime(row['Transaction Date'])

        for offset in days_offsets:
            col_name = f'Price_{offset}_days'
            if pd.isna(row[col_name]):
                offset_date = transaction_date + pd.Timedelta(days=offset)
                if offset_date <= today:
                    closest_price = fetch_closest_price(ticker_symbol, offset_date)
                    if closest_price is not None:
                        df.at[index, col_name] = closest_price
                    # else:
                    #     print(f"No valid data found for {ticker_symbol} {offset_date.date()}")

    df.to_csv(file_path, index=False)

process_data()
print("Price updates complete.")

Processing rows:   9%|▊         | 765/8989 [13:38<2:41:30,  1.18s/it]ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KD']: Exception("%ticker%: Data doesn't exist for startDate = 1632196800, endDate = 1632715200")
Processing rows:   9%|▉         | 795/8989 [14:19<2:53:48,  1.27s/it]ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KD']: Exception("%ticker%: Data doesn't exist for startDate = 1632196800, endDate = 1632715200")
Processing rows:  12%|█▏        | 1081/8989 [19:24<2:55:09,  1.33s/it]ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['RIVN']: Data doesn't exist for startDate = 1632196800, endDate = 1632715200
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['RIVN']: Exception("%ticker%: Data doesn't exist for startDate = 1632196800, endDate = 1632715200")
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['RIVN']: Exception("%ticker%: Data doesn't exist for startDate = 1634788800, endDate = 1635307200")
Processing rows:  12%|█▏        | 1082/8989 [19:27<4:02:14,  1.8

Price updates complete.


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31589 entries, 0 to 31588
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Politician Name            31589 non-null  object        
 1   Party                      31589 non-null  object        
 2   Chamber                    31589 non-null  object        
 3   State                      31589 non-null  object        
 4   Issuer Name                31589 non-null  object        
 5   Ticker                     31589 non-null  object        
 6   Publication Date           31589 non-null  object        
 7   Transaction Date           31589 non-null  datetime64[ns]
 8   Reporting Gap              31589 non-null  int64         
 9   Owner                      31589 non-null  object        
 10  Transaction Type           31589 non-null  object        
 11  Value Range                31589 non-null  object        
 12  Pric

In [5]:
df.tail()

,Politician Name,Party,Chamber,State,Issuer Name,Ticker,Publication Date,Transaction Date,Reporting Gap,Owner,...,Price_-90_days,Price_-60_days,Price_-40_days,Price_-20_days,Price_-10_days,Price_10_days,Price_20_days,Price_40_days,Price_60_days,Price_90_days
31584,Alan Lowenthal,Democrat,House,CA,Americold Realty Trust,COLD,2021-03-24,2021-03-15,8,Spouse,...,34.84,34.180000,36.070000,37.450001,34.259998,38.180000,38.990002,39.259998,38.360001,39.270000
31585,Mark Green,Republican,House,TN,NGL Energy Partners LP,NGL,2021-03-21,2021-03-17,3,Undisclosed,...,2.78,2.900000,2.660000,2.530000,2.550000,2.100000,2.210000,2.090000,2.230000,2.270000
31586,Doug Lamborn,Republican,House,CO,NetApp Inc,NTAP,2021-03-20,2021-03-19,0,Spouse,...,64.82,64.279999,69.730003,62.599998,64.610001,72.400002,73.680000,76.599998,77.279999,81.470001
31587,Doug Lamborn,Republican,House,CO,NetApp Inc,NTAP,2021-03-20,2021-03-19,0,Undisclosed,...,64.82,64.279999,69.730003,62.599998,64.610001,72.400002,73.680000,76.599998,77.279999,81.470001
31588,Doug Lamborn,Republican,House,CO,NetApp Inc,NTAP,2021-03-20,2021-03-19,0,Spouse,...,64.82,64.279999,69.730003,62.599998,64.610001,72.400002,73.680000,76.599998,77.279999,81.470001
